In [1]:
import json
import csv
import time
from datetime import datetime, timedelta
from numpy import array
import numpy as np
import pandas as pd


In [15]:
symbols = ["OTO", "HPT", "AMANAH"]
# items = ["2017-02-14", "2017-02-15"]
items = ["2017-02-14"]


In [7]:
def getFreefloat():
    allShare = 250000000  # ACAP
    freeFloat = 67.03
    return (allShare * freeFloat)/100

def check(list):
    if list[1] is not 0 and list[2] is not 0 and list[4] is 0: return 0
    else: return 1

def getSumVol(list):
    # Sum = list[0] + list[1] + list[2] + list[3] + list[4]
    Sum = np.sum(list)
    return Sum

def event(eve):
    if eve == "B":
        return 1
    elif eve == "S":
        return 0
    else:
        return 5

def getTime(time):
    timeSplit = time.split(':')
    return timeSplit

def getDate(Date):
    dateSplit = Date.split('-')
    return dateSplit


def Timestamp2Datetime(Timestamp):
    Datetime = datetime.fromtimestamp(Timestamp-25200).strftime('%Y-%m-%d-%H-%M-%S')
    # print Datetime
    return Datetime

def Datetime2Timestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    # return td.total_seconds()
    return (td.microseconds + (td.seconds + td.days * 86400) * 10**6) // 10**6


def toTemp(price, vol, eve, symbol):
    sumVO = getSumVol(vol)
    compareBid1[symbol] = tempBid[symbol][5]
    compareOff1[symbol] = tempOffer[symbol][5]


    if eve == 1:

        tempBid[symbol] = price + vol

        difBO[symbol] = sumVO - tempBidVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -4
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -3
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -2
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -1
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 1
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 2
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 3
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 4

        tempBidVol[symbol] = sumVO
        return price + vol + tempOffer[symbol] + [tempBidVol[symbol]] + [tempOffVol[symbol]]
    else:

        tempOffer[symbol] = price + vol

        difBO[symbol] = sumVO - tempOffVol[symbol]

        if difBO[symbol] <= -1000000:
            volBO[symbol] = -40
        elif -1000000 < difBO[symbol] <= -500000:
            volBO[symbol] = -30
        elif -500000 < difBO[symbol] <= -100000:
            volBO[symbol] = -20
        elif -100000 < difBO[symbol] <= -100:
            volBO[symbol] = -10
        elif -100 < difBO[symbol] <= 100000:
            volBO[symbol] = 10
        elif 100000 < difBO[symbol] <= 500000:
            volBO[symbol] = 20
        elif 500000 < difBO[symbol] <= -1000000:
            volBO[symbol] = 30
        elif difBO[symbol] > 1000000:
            volBO[symbol] = 40

        tempOffVol[symbol] = sumVO
        return tempBid[symbol] + price + vol + [tempBidVol[symbol]] + [tempOffVol[symbol]]



def checkTimestamp(timestamp,symbol):
    if timestamp < tempTimestamp[symbol]:
        return tempTimestamp[symbol],1
    else:
        tempTimestamp[symbol] = timestamp
        return timestamp,0










def getSpread(x):
    ''' x<2         0.01    0
        2<=x<5       0.02    1
        5<=x<10      0.05    2
        10<=x<25     0.1     3
        25<=x<100    0.25    4
        100<=x<200   0.5     5
        200<=x<400   1.0     6
        x>=400       2.0     7
    '''
    if x < 200:
        return 0, 1, 200
    elif 200 <= x < 500:
        return 1, .5, 500
    elif 500 <= x < 1000:
        return 2, .2, 1000
    elif 1000 <= x < 2500:
        return 3, .1, 2500
    elif 2500 <= x < 10000:
        return 4, .04, 10000
    elif 10000 <= x < 20000:
        return 5, .02, 20000
    elif 20000 <= x < 40000:
        return 6, .01, 40000
    else:
        return 7, .005, 40000

def getDifSpread(x1, x2):
    a1, t1, z1 = getSpread(x1)
    a2, t2, z2 = getSpread(x2)
    if a1 == a2:
        Dif = (x2 - x1) * t1
    elif a2 > a1:
        Dif = ((x2 - z1) * t2) + ((z1 - x1) * t1)
    else:
        Dif = -(((x1 - z2) * t1) + ((z2 - x2) * t2))
    return int(Dif)



def calmoney(pbid0,pbid1,pbid2,pbid3,pbid4,vbid0,vbid1,vbid2,vbid3,vbid4):
    money = ((pbid0*vbid0)+(pbid1*vbid1)+(pbid2*vbid2)+(pbid3*vbid3)+(pbid4*vbid4))/100
    return money

In [14]:
idSymbol = dict()
tempBid = dict()
tempOffer = dict()
tempBidVol = dict()
tempOffVol = dict()
tempAll = dict()
priATO = dict()
volATO = dict()
tempTimestamp = dict()
marketStatus = dict()
forTemp = dict()
clearNoise = dict()

sumOrder = dict()
countOrder = dict()

lastPrice = dict()
prior = dict()
highPrice = dict()
lowPrice = dict()
avgPrice = dict()
tradeVol = dict()
buyVol = dict()
sellVol = dict()
auctVol = dict()

spread = dict()
count100k = dict()
difBO = dict()

tf30 = dict()
volBO = dict()
typeBO = dict()
compareBid1 = dict()
compareOff1 = dict()


allbidmoney = dict()
sidetransac = dict()



stock =dict()


for symbol in symbols:
    # tempBid[symbol] = [None, None, None, None, None, None, None, None, None, None]
    # tempOffer[symbol] = [None, None, None, None, None, None, None, None, None, None]
    tempBid[symbol] = [0,0,0,0,0,0,0,0,0,0]
    tempOffer[symbol] = [0,0,0,0,0,0,0,0,0,0]
    # forTemp[symbol] = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

    priATO[symbol] = 0
    volATO[symbol] = 0
    tempTimestamp[symbol] = 0
    marketStatus[symbol] = 0
    clearNoise[symbol] = 0
    idSymbol[symbol] = 0

    sumOrder[symbol] = 0
    countOrder[symbol] = 0

    lastPrice[symbol] = 0
    prior[symbol] = 0
    highPrice[symbol] = 0
    lowPrice[symbol] = 0
    avgPrice[symbol] = 0
    tradeVol[symbol] = 0
    buyVol[symbol] = 0
    sellVol[symbol] = 0
    auctVol[symbol] = 0

    spread[symbol] = 0
    count100k[symbol] = 0
    difBO[symbol] =0
    tf30[symbol] = 0
    volBO[symbol] = 0
    typeBO[symbol] = 0

    tempBidVol[symbol] = 0
    tempOffVol[symbol] = 0
    compareBid1[symbol] = 0
    compareOff1[symbol] = 0

    allbidmoney[symbol] = 0
    sidetransac[symbol] = 0
    
    stock[symbol] = pd.DataFrame()
    


for symbol in symbols:
    forTemp[symbol] = []


In [16]:
t0 = time.time()

ref_files = [open("TESTIPython\\List\\" + Symbol + ".csv", "a") for Symbol in symbols]
ref_files2 = [open("TESTIPython\\Listato\\" + Symbol + ".csv", "a") for Symbol in symbols]


for Date in items:

    YYYY,MM,DD = getDate(Date)

    YYYY = int(YYYY)
    MM = int(MM)
    DD = int(DD)
    
    with open("INPUT\\" +Date+".dat") as f:
        with open("TESTWRITE\\ma1" + items[0] + "to" + items[-1] + ".csv", "a") as output: 
            
            content = f.readlines()
            for line in content:
                if 'data' in line:
                    try:
                        jsonData = line.replace("data: ", "")
                        jsonDecoded = json.loads(jsonData)
                    except ValueError:
                        # print Date, jsonData
                        continue

                    if 'sym' in jsonDecoded.keys():
                        sym = jsonDecoded["sym"]
                        if sym in symbols:
                            if 'isf' in jsonDecoded.keys():
                                
                                pri = jsonDecoded["pri"]  # price
                                priATO[sym] = pri

                                tim = jsonDecoded["tim"]    #time
                                HH, mm, SS = getTime(jsonDecoded["tim"])            ###
                                HH = int(HH)
                                mm = int(mm)
                                SS = int(SS)
                                Timestamp = Datetime2Timestamp(datetime(YYYY, MM, DD, HH, mm, SS))  ###
                                Timestamp = checkTimestamp(Timestamp,sym)

                                vol = jsonDecoded["vol"]  # volume
                                volATO[sym] = vol

                                op1 = jsonDecoded["op1"]  # open1
                                op2 = jsonDecoded["op2"]  # open2

                                isf = jsonDecoded["isf"]
                                if isf == 'F':
                                    marketStatus[sym] = 0   #Pre-Open1
                                    if HH == 14:
                                        marketStatus[sym] = 2   #Pre-Open2
                                    if HH == 16:
                                        marketStatus[sym] = 3  # Pre-Open2

                                elif isf == 'T':
                                    marketStatus[sym] = 1   #Open
                                    clearNoise[sym] = Timestamp[0]

                                    if HH == 16 and mm >30:
                                        marketStatus[sym] = 4   #Pre-Close
                                else:
                                    marketStatus[sym] = 5
                                    print "Unknown status"
                                    
                                    
                                
#                                 table = [marketStatus[sym]] + [Timestamp[0]] + [Date] + [tim] + [pri] + [vol] + [op1] + [op2]
#                                 df2 = pd.DataFrame(table)
#                                 stock[sym] = stock[sym].append(df2, ignore_index=True)
                                
                                stock[sym] = stock[sym].append({'marketStatus':marketStatus[sym], 'Timestamp':Timestamp[0], 'Date':Date, 'time':tim, 'pri':pri, 'vol':vol, 'op1':op1, 'op2':op2}, ignore_index=True)
                                
                                
                                
                                
t1 = time.time()
total = t1-t0

print total
                                


60.493999958


In [17]:
stock['AMANAH']

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0
5,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,267900.0
6,2017-02-14,1.487065e+09,0.0,0.0,0.0,153.0,09:30:23,267900.0
7,2017-02-14,1.487065e+09,0.0,0.0,0.0,150.0,09:30:28,267900.0
8,2017-02-14,1.487065e+09,0.0,0.0,0.0,148.0,09:30:28,267900.0
9,2017-02-14,1.487065e+09,0.0,0.0,0.0,147.0,09:30:29,267900.0


In [33]:
stock1 = stock


In [44]:
stock1['AMANAH'].head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0


In [42]:
stock1['AMANAH'].assign(ln_A = lambda x: np.log(x.vol)).head()

,Date,Timestamp,marketStatus,op1,op2,pri,time,vol,ln_A
0,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:21,221800.0,12.309531
1,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,231800.0,12.353630
2,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,241800.0,12.395866
3,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:22,251800.0,12.436390
4,2017-02-14,1.487065e+09,0.0,0.0,0.0,182.0,09:30:23,261800.0,12.475336
